In [1]:
# =============================================================================
# IMPORTS
# =============================================================================

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✓ Imports loaded")

✓ Imports loaded


In [2]:
# =============================================================================
# DATA LOADING (from Notebook 1)
# =============================================================================

BASKETS = {
    'cybersecurity': ['CRWD', 'PANW', 'ZS', 'FTNT', 'OKTA', 'S', 'VRNS', 'TENB', 'CYBR', 'NET'],
    'uranium': ['UUUU', 'UEC', 'CCJ', 'LEU', 'DNN', 'NXE', 'URG', 'SMR'],
    'quantum': ['IONQ', 'RGTI', 'QBTS'],
    'space': ['RKLB', 'ASTS', 'LUNR'],
    'semi': ['NVTS', 'NXPI', 'SWKS', 'MRVL', 'ARM', 'NVDA', 'AMD', 'INTC', 'MU', 'TSM'],
}

START_DATE = '2024-01-01'
END_DATE = '2026-01-08'

def load_basket_data(tickers, start=START_DATE, end=END_DATE):
    data = {}
    for ticker in tickers:
        try:
            df = yf.download(ticker, start=start, end=end, progress=False)
            if len(df) > 0:
                df['Returns'] = df['Adj Close'].pct_change()
                df['Ticker'] = ticker
                data[ticker] = df
                print(f"✓ {ticker}: {len(df)} days")
        except Exception as e:
            print(f"✗ {ticker}: {e}")
    return data

def load_sector(sector_name):
    if sector_name in BASKETS:
        print(f"\n📦 Loading {sector_name.upper()}")
        return load_basket_data(BASKETS[sector_name])
    return {}

print("✓ Functions ready")

✓ Functions ready


In [3]:
# =============================================================================
# CONFIGURATION - EDIT THIS
# =============================================================================

# SECTOR TO TEST
SECTOR = 'cybersecurity'

# MOVE THRESHOLD (% move to count as "participating")
MOVE_THRESHOLD = 0.03  # 3% move

# MINIMUM COORDINATION (minimum stocks moving together)
MIN_COORDINATION = [3, 4, 5, 6, 7, 8]

# FORWARD PERIODS TO TEST
FORWARD_PERIODS = [1, 2, 3, 5, 10]

print("✓ Configuration set")
print(f"  Sector: {SECTOR}")
print(f"  Move threshold: {MOVE_THRESHOLD*100:.0f}%")
print(f"  Min coordination: {MIN_COORDINATION}")
print(f"  Forward periods: {FORWARD_PERIODS} days")

✓ Configuration set
  Sector: cybersecurity
  Move threshold: 3%
  Min coordination: [3, 4, 5, 6, 7, 8]
  Forward periods: [1, 2, 3, 5, 10] days


In [4]:
# =============================================================================
# COORDINATION DETECTION
# =============================================================================

def calculate_daily_coordination(sector_data, threshold):
    """
    For each day, count how many stocks moved > threshold in same direction.
    """
    # Build returns matrix
    returns_dict = {}
    for ticker, df in sector_data.items():
        returns_dict[ticker] = df['Returns']
    
    returns_df = pd.DataFrame(returns_dict)
    
    # Count stocks moving up > threshold
    up_count = (returns_df > threshold).sum(axis=1)
    
    # Count stocks moving down > threshold  
    down_count = (returns_df < -threshold).sum(axis=1)
    
    # Total stocks with data that day
    total_stocks = returns_df.notna().sum(axis=1)
    
    coordination = pd.DataFrame({
        'up_count': up_count,
        'down_count': down_count,
        'total_stocks': total_stocks,
        'up_pct': up_count / total_stocks,
        'down_pct': down_count / total_stocks,
        'coordination_score': up_count - down_count,  # positive = bullish coordination
        'sector_return': returns_df.mean(axis=1)  # average return of sector
    })
    
    return coordination

print("✓ Coordination detection ready")

✓ Coordination detection ready


In [5]:
# =============================================================================
# SIGNAL ANALYSIS
# =============================================================================

def analyze_coordination_signals(coordination_df, sector_data, min_coord, forward_periods):
    """
    When coordination score >= min_coord, what happens next?
    """
    # Get average sector price for forward return calculation
    prices_dict = {}
    for ticker, df in sector_data.items():
        prices_dict[ticker] = df['Adj Close']
    prices_df = pd.DataFrame(prices_dict)
    avg_price = prices_df.mean(axis=1)
    
    results = []
    
    # Find high coordination days
    high_coord_days = coordination_df[coordination_df['up_count'] >= min_coord]
    
    for date, row in high_coord_days.iterrows():
        if date in avg_price.index:
            idx = avg_price.index.get_loc(date)
            entry_price = avg_price.iloc[idx]
            
            for period in forward_periods:
                if idx + period < len(avg_price):
                    exit_price = avg_price.iloc[idx + period]
                    forward_return = (exit_price - entry_price) / entry_price
                    
                    results.append({
                        'date': date,
                        'up_count': row['up_count'],
                        'coordination_score': row['coordination_score'],
                        'sector_return_that_day': row['sector_return'],
                        'forward_period': period,
                        'forward_return': forward_return,
                        'win': forward_return > 0
                    })
    
    return pd.DataFrame(results)

print("✓ Signal analysis ready")

✓ Signal analysis ready


In [6]:
# =============================================================================
# FULL COORDINATION BACKTEST
# =============================================================================

def backtest_coordination(sector_name, thresholds=[0.02, 0.03], 
                          min_coords=MIN_COORDINATION,
                          forward_periods=FORWARD_PERIODS):
    """
    Test all combinations.
    """
    sector_data = load_sector(sector_name)
    
    if len(sector_data) == 0:
        print("No data loaded")
        return pd.DataFrame()
    
    all_results = []
    
    print(f"\n🔍 Testing {len(thresholds)} thresholds × {len(min_coords)} coordination levels")
    print("="*60)
    
    for threshold in thresholds:
        print(f"\nMove threshold: {threshold*100:.0f}%")
        
        coordination = calculate_daily_coordination(sector_data, threshold)
        
        for min_coord in min_coords:
            results = analyze_coordination_signals(
                coordination, sector_data, min_coord, forward_periods
            )
            
            if len(results) > 0:
                results['move_threshold'] = threshold
                results['min_coordination'] = min_coord
                all_results.append(results)
                
                win_rate = results['win'].mean()
                avg_return = results['forward_return'].mean()
                num_signals = len(results['date'].unique())
                print(f"  Min {min_coord} stocks: {num_signals} signals, "
                      f"{win_rate*100:.1f}% win rate, {avg_return*100:.2f}% avg return")
    
    if all_results:
        return pd.concat(all_results, ignore_index=True)
    return pd.DataFrame()

print("✓ Backtest engine ready")

✓ Backtest engine ready


In [7]:
# =============================================================================
# EXECUTE - RUN THE BACKTEST
# =============================================================================

print("\n🐺 STARTING COORDINATION BACKTEST")
print("="*60)

coord_results = backtest_coordination(SECTOR, thresholds=[0.02, 0.03])

# Best parameters
if len(coord_results) > 0:
    coord_summary = coord_results.groupby(
        ['move_threshold', 'min_coordination', 'forward_period']
    ).agg({
        'forward_return': 'mean',
        'win': 'mean',
        'date': 'count'
    }).rename(columns={'date': 'num_signals'})
    
    coord_summary = coord_summary.reset_index()
    coord_summary['expected_value'] = coord_summary['forward_return'] * coord_summary['win']
    
    print("\n" + "="*60)
    print("📊 COORDINATION STRATEGY RESULTS")
    print("="*60)
    print(coord_summary.sort_values('expected_value', ascending=False).head(20).to_string(index=False))
    
    # Best setup
    best = coord_summary.sort_values('expected_value', ascending=False).iloc[0]
    print(f"\n🎯 BEST SETUP (by expected value):")
    print(f"   Move threshold: {best['move_threshold']*100:.0f}%")
    print(f"   Min coordination: {int(best['min_coordination'])} stocks")
    print(f"   Forward period: {int(best['forward_period'])} days")
    print(f"   Win rate: {best['win']*100:.1f}%")
    print(f"   Avg return: {best['forward_return']*100:.2f}%")
    print(f"   Expected value: {best['expected_value']*100:.2f}%")
    print(f"   Number of signals: {int(best['num_signals'])}")
else:
    print("\n✗ No results found")


🐺 STARTING COORDINATION BACKTEST

📦 Loading CYBERSECURITY
✗ CRWD: 'Adj Close'
✗ PANW: 'Adj Close'
✗ ZS: 'Adj Close'
✗ FTNT: 'Adj Close'
✗ OKTA: 'Adj Close'
✗ S: 'Adj Close'
✗ VRNS: 'Adj Close'
✗ TENB: 'Adj Close'
✗ CYBR: 'Adj Close'
✗ NET: 'Adj Close'
No data loaded

✗ No results found


In [8]:
# =============================================================================
# VISUALIZATION
# =============================================================================

if len(coord_results) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'🐺 COORDINATION ANALYSIS: {SECTOR.upper()}', fontsize=16, fontweight='bold')
    
    # Win rate by coordination level
    for period in [1, 3, 5]:
        data = coord_summary[coord_summary['forward_period'] == period]
        axes[0, 0].plot(data['min_coordination'], data['win'], 
                        marker='o', label=f'{period}-day', linewidth=2)
    axes[0, 0].set_xlabel('Minimum Coordination (# stocks)', fontsize=12)
    axes[0, 0].set_ylabel('Win Rate', fontsize=12)
    axes[0, 0].set_title('Win Rate by Coordination Level', fontweight='bold')
    axes[0, 0].legend(title='Forward Period')
    axes[0, 0].axhline(y=0.5, color='r', linestyle='--', alpha=0.5)
    axes[0, 0].grid(alpha=0.3)
    
    # Average return by coordination level
    for period in [1, 3, 5]:
        data = coord_summary[coord_summary['forward_period'] == period]
        axes[0, 1].plot(data['min_coordination'], data['forward_return']*100, 
                        marker='o', label=f'{period}-day', linewidth=2)
    axes[0, 1].set_xlabel('Minimum Coordination (# stocks)', fontsize=12)
    axes[0, 1].set_ylabel('Average Return (%)', fontsize=12)
    axes[0, 1].set_title('Average Return by Coordination Level', fontweight='bold')
    axes[0, 1].legend(title='Forward Period')
    axes[0, 1].axhline(y=0, color='r', linestyle='--', alpha=0.5)
    axes[0, 1].grid(alpha=0.3)
    
    # Signal frequency by coordination level
    signal_freq = coord_summary.groupby('min_coordination')['num_signals'].mean()
    axes[1, 0].bar(signal_freq.index, signal_freq.values, color='steelblue', alpha=0.7)
    axes[1, 0].set_xlabel('Minimum Coordination (# stocks)', fontsize=12)
    axes[1, 0].set_ylabel('Average Signals', fontsize=12)
    axes[1, 0].set_title('Signal Frequency', fontweight='bold')
    axes[1, 0].grid(alpha=0.3, axis='y')
    
    # Expected value heatmap
    pivot = coord_summary.pivot_table(values='expected_value', 
                                      index='min_coordination', 
                                      columns='forward_period', 
                                      aggfunc='mean')
    im = axes[1, 1].imshow(pivot.values, cmap='RdYlGn', aspect='auto')
    axes[1, 1].set_xticks(range(len(pivot.columns)))
    axes[1, 1].set_xticklabels(pivot.columns)
    axes[1, 1].set_yticks(range(len(pivot.index)))
    axes[1, 1].set_yticklabels([f"{int(x)}" for x in pivot.index])
    axes[1, 1].set_xlabel('Forward Period (days)', fontsize=12)
    axes[1, 1].set_ylabel('Min Coordination', fontsize=12)
    axes[1, 1].set_title('Expected Value Heatmap', fontweight='bold')
    plt.colorbar(im, ax=axes[1, 1])
    
    plt.tight_layout()
    plt.show()
else:
    print("No data to visualize")

No data to visualize


---

## 📊 INTERPRETATION

**What to look for:**
- **Win rate increasing with coordination** = signal works (more movers = stronger)
- **Sweet spot coordination level** = too low (noisy), too high (rare)
- **1-day forward best** = fast follow-through (momentum)
- **5-day forward best** = sustained continuation (rotation)

**From Jan 2-6 validation:**
- 14 movers = mega signal (Day 1)
- 8 movers = strong signal (Day 2-3 continuation)
- <5 movers = party ending

**Next steps:**
1. Compare to morning_decision.py thresholds (8+ movers)
2. Test on uranium, quantum, space sectors
3. Combine with leader/laggard for entry selection

🐺 **The herd moves together. Count the buffalo.**